# Notebook 4: API para probar el modelo de Azure

In [20]:
import pandas as pd

# Cargar los datos de prueba desde la carpeta de prueba
data = pd.read_csv("../data/prueba.csv").drop(["Unnamed: 0"], axis=1)
data2=data.copy()
#Retirar la primera columna que es un índice no deseado, dado que es la variable objetivo
data = data.drop(data.columns[0], axis=1)
# Convertir el DataFrame a un diccionario
data_dict = data.to_dict(orient='list')

data

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,302583.846829,215926.668338,160681.375234,113789.973083,363023.506580,21337.201274,391837.887305,401233.992710,-3081.409915,339932.913561,...,-20803.840865,-31727.661064,254483.525085,91398.771828,185869.684595,56136.475541,6439.213877,194339.999725,218748,-21751.619260
1,296828.273778,213045.320105,155028.177793,111604.727814,357377.710979,21333.981513,391764.751701,401162.054605,-3081.382844,339868.922278,...,-20816.707256,-32506.279096,251459.324081,89736.384811,186528.801063,63877.101621,6446.586494,194383.053127,218748,-30299.884588
2,327271.457125,234716.999128,172092.136838,114193.628800,364066.390738,21337.086282,392031.534720,401360.048787,-3074.991683,340061.212601,...,-20231.915888,-30668.672365,251464.519612,91705.142213,186246.487611,55978.622787,6426.802673,194262.498897,218748,-18209.737872
3,269715.726612,192432.598125,142886.849135,109683.693560,352414.530463,21329.381856,391660.207706,401078.740547,-3081.355773,339795.500035,...,-21387.657854,-27070.202705,251224.732013,88276.137513,185237.572268,58382.352910,6204.241904,187676.849320,218748,-29077.210429
4,322100.206211,225137.131923,173448.904224,113136.784740,361335.930397,21336.807017,391821.994810,401217.421958,-3081.631403,339927.723368,...,-20465.789236,-32166.781557,250142.316882,90901.039680,186230.436500,56715.036089,6434.530874,194311.528008,218748,-26386.100504
5,287800.496758,205263.217183,150540.408748,111140.523739,356173.653815,21333.114149,391748.557926,401150.086840,-3081.267178,339858.162122,...,-20915.443338,-28434.561754,251408.065819,89383.825463,185875.550424,57709.995625,6441.210105,194351.866011,218748,-28496.495732
6,289348.950422,207184.116005,153619.227046,112415.708846,359472.960060,21334.106361,391770.174732,401170.076953,-3081.301632,339880.188795,...,-20859.146789,-22364.276630,251398.005486,90352.769763,185871.470625,56714.641320,6442.092860,194357.063173,218748,-26384.535195
7,252361.359138,181128.847362,133615.605332,111678.119762,357567.326280,21332.325637,391628.949955,401066.707025,-3084.582116,339766.067843,...,-21519.365535,-32513.768574,251241.135760,89792.593103,185154.893963,57580.732969,6396.280314,194039.800334,218748,-28340.347833
8,344070.718569,233264.011557,181311.197280,116030.262313,368124.158189,21339.918357,391902.963683,401261.742144,-3081.318859,339967.599241,...,-20298.628357,-26092.502522,238024.075425,93102.294268,186386.240393,56728.261505,6427.416966,194266.493014,218748,-26437.924626
9,282191.004241,203982.617968,150679.564377,114184.454806,364042.688825,21331.116584,391896.109574,401297.711196,-3074.944925,340042.477270,...,-20934.411461,-29021.142846,251018.642896,91698.156767,185792.157626,56500.961406,6460.832308,194460.613492,218748,-25342.485140


## Cargar aquí el json con el url que dirige al modelo, se debe encontrar en la misma carpeta de src

In [21]:
import json

#Formateado para la API:
data_json = json.dumps({"data": [data_dict]})

suri = open("uri2.json", "r")
scoring_uri = json.load(suri)["URI"][0]
suri.close()

Estructura de la API

In [22]:


headers = {"Content-Type": "application/json"}
import requests
response = requests.post(scoring_uri, data=data_json, headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print('Predicciones del modelo:',result)
  data["Exited"] = result
  print('Variable objetivo real:',list(data2['Bankrupt?']))
  #Print la precisión del modelo usando el porcentaje de aciertos
  aciertos = 0
  for i in range(len(data)):
    if data["Exited"][i] == list(data2['Bankrupt?'])[i]:
      aciertos += 1
  precision = aciertos / len(data) * 100
  print(f"Precisión del modelo: {precision:.2f}%")
else:
  print(f"Error: {response.text}")

Predicciones del modelo: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Variable objetivo real: [0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
Precisión del modelo: 80.00%


# Conclusión

El modelo de Regresión Logística subido a la nube de azure está sesgado, dado que las clases no están balanceadas, por lo que, si se predice que la mayoría del tiempo se predice `Bankrupt?` como negativa, casi siempre se tendrá una predicción correcta

Este ejercicio representa un experimento para subir modelos de ML a la nubes y utilizarlos en un 'entorno de producción', sin embargo el modelo utilizado está lejos del óptimo para atacar este problema adecuadamente